In [45]:
%load_ext autoreload
%autoreload 2

# For debugging turn on logging to console
import logging
logging.basicConfig(level=logging.DEBUG)
log = logging.getLogger(__name__)
logging.getLogger('urllib3').setLevel(logging.WARNING)


# mongodb
import pymongo as pm

# fine-tuned newspaper lib
from resources.newspaper import newspaper
from resources.newspaper.newspaper.source import Source
from resources.newspaper.newspaper.article import Article

import bs4 as bs
from urllib.parse import urljoin
from dateutil.parser import parse as date_parser
from time import sleep
import random
import pytz
from datetime import datetime, timedelta

import requests

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pm.MongoClient(conn, maxPoolSize=200)

# define db 
# DB_NAME = 'scrape'
DB_NAME = 'FINALP'
db = client[DB_NAME]

def saveToDB(db, collection, url, html, meta={}):
    """
    Saves a document to mongoDB, making sure there are no duplicates by 
    'url' value
    
    Parameters:
    --------
    db, collection  : mongo db connection
    url, html, meta : values to store
    
    Returns:
    --------
    Saved document
    """
    collection = db[collection]
    collection.update_one(
        {'url' : url},
        {
            '$set':
                {'url' : url,
                 'html' : html,
                 'meta' : meta
                }
        }
        ,
        upsert=True
    )
    log.debug(f'Saved to DB')

def scrape(url, db, collection):
    '''
    Scrapes an article from the 'url' up to the 'latest_date'
    
    Parameters:
    --------
    url         : main news website url
    date        : YYYY-MM-DD
    db          : database name
    collection  : mongodb collection
    
    Returns:
    --------
    Article's html and features stored to db, 
    Article's publish date
    
    '''
    log.debug(f"Exctracting features from {url}")
    try:
        article = Article(url)
        article.download()
        # the below method may only extract a snippet... 
        # check the database for results of text extraction
        # and apply additional processing if needed after 
        # article has been stored in the DB
        # see code below Newrepublic for example
        article.parse()
    except Exception as e:
        log.critical(f'Data not saved: {e}')
        return datetime.datetime.now()
    
    saveToDB(db, collection, article.url, article.html, meta={
        'date'    :article.publish_date,
        'title'   :article.title,
        'text'    :article.text,
        'authors' :article.authors
    })
    
    return article.publish_date

def addToDB(DB_NAME,COL_NAME,PATH,FILE):
    '''
    Imports a file into mongoDB
    
    Parameters:
    --------
    DB_NAME : Name of the database to connect to
    COL_NAME: Name of the collection to create
    PATH    : Path to folder with the file
    FILE  : Filename
    
    Returns:
    --------
    Collection COL_NAME in DB_NAME database
    '''
    !mongoimport --db {DB_NAME} --collection {COL_NAME} --file {PATH+FILE} --batchSize 1
    print(f'Collection {COL_NAME} in {DB_NAME} database created')

In [ ]:
collection = 'huffpost'
source = 'https://www.huffingtonpost.com/archive/'
earliest_date = date_parser('2017-01-01')
latest_date = date_parser('2017-05-10')
stopWords = ['HEALTHY LIVING',
            'ENTERTAINMENT',
            'STYLE',
            'COMEDY',
            'WEDDINGS',
            'SPORTS',
            'ARTS & CULTURE',
            'TASTE',
            'PARENTS',
            'BOOKS',
            'HUFFPOST PERSONAL',
            'QUEER VOICES']

base_url = 'https://www.huffingtonpost.com/'
date = 1

while True:
    date_to_scrape = str(latest_date-timedelta(date))[:10]
    log.debug(f"==================\n Processing DATE {date_to_scrape}\n URL: {source+date_to_scrape}")
    s = Source(source+date_to_scrape)
    date += 1
    s.download()
    soup = bs.BeautifulSoup(s.html,'lxml')
    for div in soup.findAll('div', {'class':'card__details'}):
        try:
            category = div.span.text
            if category not in stopWords:
                for a in div.findAll('a',{'class':'card__link yr-card-headline'}):
                    try:
                        url = urljoin(base_url, a['href'])
                        log.debug(f"Processing url: {url}")
                        article_date = scrape(url, db, collection)
                    except Exception as e:
                        log.debug(e)
            else:
                log.debug(f'Skipping category {category}')
        except Exception as e:
            log.debug(e)
    
    if (latest_date-timedelta(date)) <= earliest_date:
        log.debug(f"Finished. Last date scraped: {latest_date-timedelta(date)}")
        break

DEBUG:__main__:==================
 Processing DATE 2017-05-09
 URL: https://www.huffingtonpost.com/archive/2017-05-09
DEBUG:__main__:Skipping category QUEER VOICES
DEBUG:__main__:Processing url: https://www.huffingtonpost.com/entry/san-francisco-receives-record-100-million-to-fight-homelessness_us_5911400ee4b0d5d9049f6aad
DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/san-francisco-receives-record-100-million-to-fight-homelessness_us_5911400ee4b0d5d9049f6aad
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.huffingtonpost.com/entry/dear-white-people-dear-white-people-being-ethnic-is-cool-being-ethnic-is-cool_us_59121da4e4b0a58297e02b95
DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/dear-white-people-dear-white-people-being-ethnic-is-cool-being-ethnic-is-cool_us_59121da4e4b0a58297e02b95
DEBUG:__main__:Saved to DB
DEBUG:__main__:Skipping category PARENTS
DEBUG:__main__:Processing url: https://www.huffington

DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/emmanuel-macron-macarons_us_5911b5ede4b0e7021e9b1a27
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.huffingtonpost.com/entry/paul-ryan-school-visit-harlem-success-academy-protests_us_59123326e4b05e1ca202cc78
DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/paul-ryan-school-visit-harlem-success-academy-protests_us_59123326e4b05e1ca202cc78
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.huffingtonpost.com/entry/anti-muslim-hate-crimes-2016-council-on-american-islamic-relations_us_5910acf4e4b0d5d9049e96d5
DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/anti-muslim-hate-crimes-2016-council-on-american-islamic-relations_us_5910acf4e4b0d5d9049e96d5
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.huffingtonpost.com/entry/latinx-show-their-pride-and-resilience-with-spectacular-grad-caps_us_591

DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/9-mothers-day-gifts-to-boost-your-moms-tech-cessory-game_us_590a2819e4b02655f843b503
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.huffingtonpost.com/entry/4-business-mistakes-ill-never-make-again_us_5911ec18e4b05e1ca202299a
DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/4-business-mistakes-ill-never-make-again_us_5911ec18e4b05e1ca202299a
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.huffingtonpost.com/entry/gloria-steinem-sva-commencement_us_5911f393e4b05e1ca2023d4b
DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/gloria-steinem-sva-commencement_us_5911f393e4b05e1ca2023d4b
DEBUG:__main__:Saved to DB
DEBUG:__main__:Skipping category SPORTS
DEBUG:__main__:Skipping category COMEDY
DEBUG:__main__:Skipping category HEALTHY LIVING
DEBUG:__main__:Skipping category WEDDINGS
DEBUG:__main__:Skipping category PAR

DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/all-you-wanna-do-is-ride-around-sally-and-expose-malfeasance_us_5910d9fbe4b0d5d9049f205b
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.huffingtonpost.com/entry/jordan-peeles-get-out-make-history_us_5910988ce4b0104c7350e65e
DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/jordan-peeles-get-out-make-history_us_5910988ce4b0104c7350e65e
DEBUG:__main__:Saved to DB
DEBUG:__main__:Skipping category ENTERTAINMENT
DEBUG:__main__:Processing url: https://www.huffingtonpost.com/entry/obama-profile-in-courage-speech_us_590fcef1e4b0104c73500640
DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/obama-profile-in-courage-speech_us_590fcef1e4b0104c73500640
DEBUG:__main__:Saved to DB
DEBUG:__main__:Skipping category COMEDY
DEBUG:__main__:Processing url: https://www.huffingtonpost.com/entry/warren-buffett-health-care_us_5910790fe4b0e7021e994ade
DEBU

DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.huffingtonpost.com/entry/a-victory-for-macron-and-for-the-european-union-now_us_590fd1e7e4b046ea176aecc9
DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/a-victory-for-macron-and-for-the-european-union-now_us_590fd1e7e4b046ea176aecc9
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.huffingtonpost.com/entry/in-france-neoliberalism-defeats-neofascismfor-now_us_590fe000e4b056aa2363d69c
DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/in-france-neoliberalism-defeats-neofascismfor-now_us_590fe000e4b056aa2363d69c
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.huffingtonpost.com/entry/the-vicious-knot-of-syria-the-untangling-process-contains_us_590ff9efe4b0f7118072462a
DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/the-vicious-knot-of-syria-the-untangling-process-contains_us_590ff9efe4b0f71180

...

DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/brazil-cities-paralyzed-by-nationwide-strike-against-austerity_us_590355d7e4b0bb2d086d7ff8
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.huffingtonpost.com/entry/taliban-spring-offensive_us_5902cc20e4b02655f83b5c59
DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/taliban-spring-offensive_us_5902cc20e4b02655f83b5c59
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.huffingtonpost.com/entry/chris-murphy-gun-violence-victims_us_59038c14e4b02655f83d39f2
DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/chris-murphy-gun-violence-victims_us_59038c14e4b02655f83d39f2
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.huffingtonpost.com/entry/bret-stephens-nyt_us_5903b95fe4b05c39767fa198
DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/bret-stephens-nyt_us_5903b95fe4b05c3

DEBUG:__main__:Saved to DB
DEBUG:__main__:Skipping category ENTERTAINMENT
DEBUG:__main__:Skipping category HEALTHY LIVING
DEBUG:__main__:Processing url: https://www.huffingtonpost.com/entry/charmaine-yoest-hhs_us_590370c4e4b02655f83d020d
DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/charmaine-yoest-hhs_us_590370c4e4b02655f83d020d
DEBUG:__main__:Saved to DB
DEBUG:__main__:Skipping category PARENTS
DEBUG:__main__:Processing url: https://www.huffingtonpost.com/entry/what-its-like-getting-dressed-when-youre-blind_us_58f0f747e4b0da2ff8603b01
DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/what-its-like-getting-dressed-when-youre-blind_us_58f0f747e4b0da2ff8603b01
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.huffingtonpost.com/entry/california-sea-level-rise_us_59027f0fe4b0bb2d086c5f31
DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/california-sea-level-rise_us_59027f0fe4b0bb2d

DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.huffingtonpost.com/entry/study-americans-more-likely-to-vote-for-highly-religious_us_59027b23e4b084f59b49f79c
DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/study-americans-more-likely-to-vote-for-highly-religious_us_59027b23e4b084f59b49f79c
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.huffingtonpost.com/entry/it-is-the-first-climate-lawsuit-to-make-it-this-far_us_5903396be4b084f59b49f81a
DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/it-is-the-first-climate-lawsuit-to-make-it-this-far_us_5903396be4b084f59b49f81a
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.huffingtonpost.com/entry/3-lessons-learned-from-a-31-year-old-intern_us_5903486ce4b05279d4edbb31
DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/3-lessons-learned-from-a-31-year-old-intern_us_5903486ce4b05279d4edbb31
DEBUG:_

DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/trump-100-days-private-prisons_us_590203d8e4b0026db1def8fb
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.huffingtonpost.com/entry/rachel-maddow-trump-taxes_us_59023a9ce4b0026db1df9157
DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/rachel-maddow-trump-taxes_us_59023a9ce4b0026db1df9157
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.huffingtonpost.com/entry/trumps-labor-secretary-confirmed_us_5901f3dee4b081a5c0fb5563
DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/trumps-labor-secretary-confirmed_us_5901f3dee4b081a5c0fb5563
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.huffingtonpost.com/entry/jerry-brown-trump-climate-change_us_59026ba4e4b02655f83b3b24
DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/jerry-brown-trump-climate-change_us_59026ba4e4b02655f

DEBUG:__main__:Processing url: https://www.huffingtonpost.com/entry/liberia-backstreet-abortions_us_59021196e4b0af6d718c63b8
DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/liberia-backstreet-abortions_us_59021196e4b0af6d718c63b8
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.huffingtonpost.com/entry/a-photo-shoot-fit-for-a-little-princess_us_59022883e4b0026db1df59a2
DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/a-photo-shoot-fit-for-a-little-princess_us_59022883e4b0026db1df59a2
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.huffingtonpost.com/entry/jason-chaffetz-tmz_us_59020145e4b0026db1def43b
DEBUG:__main__:Exctracting features from https://www.huffingtonpost.com/entry/jason-chaffetz-tmz_us_59020145e4b0026db1def43b
DEBUG:__main__:Saved to DB
DEBUG:__main__:Processing url: https://www.huffingtonpost.com/entry/watch-cops-narrow-rescue-of-man-jumping-from-sixth-floor_us_5901c74fe

In [66]:
log.debug(f"Stopped scraping at {date_to_scrape}")

DEBUG:__main__:Stopped scraping at 2017-05-09


In [60]:
date

54